__Домашняя работа 2.__

In [1]:
#import pip
#!pip install gensim
#!pip install razdel
#!pip install pymorphy2

In [2]:
import pandas as pd
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from razdel import tokenize
import pymorphy2 
import nltk

from gensim.models import LdaModel
from gensim.test.utils import datapath

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [4]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [9]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-8-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 28.9 s


In [10]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 3min 45s


In [11]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [12]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [13]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 45.8 s


In [14]:
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [15]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(10, 0.016902978),
 (11, 0.031029634),
 (12, 0.08514076),
 (15, 0.153681),
 (20, 0.075414136),
 (23, 0.6207929)]

In [16]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: это который россия год российский nn президент
topic_1: прибытие кг ильин химик пен монголия золотой
topic_2: газ год тело который фестиваль штат стать
topic_3: год рынок доллар цена nn спрос рейс
topic_4: фонд пенсия сообщество кость составить кожа годовой
topic_5: суд рубль преступление следствие лицо долг закон
topic_6: статья дело сотрудник выяснить гражданин задержать полиция
topic_7: год который nn млрд это также тыс
topic_8: обнаружить взрыв форум турция человек турецкий произойти
topic_9: проверка железный орден близость тереза осипов николаев
topic_10: пить этаж ирак категорически телефонный методика индонезия
topic_11: район это весь температура вода рак время
topic_12: ракета поверхность двигатель млн обращение место рейтинг
topic_13: белоруссия свидетель белорусский водитель автобус офис художественный
topic_14: экипаж собственность флот су порт пограничный лауреат
topic_15: женщина дыра прогнозировать мужчина золото женский nn
topic_16: фильм журнал мальчик реконс

In [17]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [18]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.871248,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,...,0.053156,0.0,0.0,0.000000,0.0,0.046180,0.021908,0.000000,0.000000,0.0000
1,4896,0.000000,0.0,0.0,0.0,0.317339,0.05829,0.000000,0.0,0.333731,...,0.160381,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0375
2,4897,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.085927,0.0,0.000000,...,0.163577,0.0,0.0,0.000000,0.0,0.065842,0.000000,0.000000,0.590735,0.0000
3,4898,0.075950,0.0,0.0,0.0,0.271723,0.00000,0.000000,0.0,0.000000,...,0.078053,0.0,0.0,0.000000,0.0,0.000000,0.022793,0.124436,0.417942,0.0000
4,4899,0.418672,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,...,0.079226,0.0,0.0,0.431491,0.0,0.047182,0.000000,0.000000,0.000000,0.0000


In [19]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [20]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [21]:
overall_mark = pd.DataFrame(columns=['type', 'precision', 'recall', 'f_score', 'roc auc'])
overall_mark

,type,precision,recall,f_score,roc auc


In [22]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [23]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
X = pd.merge(user_embeddings, target, 'left')
X.head(1)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.336072,0.0,0.018349,0.025412,0.023154,0.034409,0.119253,0.048078,0.036249,...,0.0,0.053373,0.027982,0.018098,0.0,0.152929,0.025779,0.0,0.02902,0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [25]:
%%time
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

Wall time: 70.9 ms


LogisticRegression()

In [26]:
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.13817816, 0.0634637 , 0.38718119, 0.24043444, 0.10200112,
       0.05872676, 0.17539397, 0.138059  , 0.23702087, 0.11053564])

In [27]:
def get_scores(y_test, preds, mtype):
    e = 1e-8 # Возможно деление на ноль
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall + e)
    ix = np.argmax(fscore)
    mark = pd.DataFrame([[mtype, precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds)]], columns=['type', 'precision', 'recall', 'f_score', 'roc auc'])
    return mark
    #overall_mark.append(mark)

In [28]:
overall_mark = overall_mark.append(get_scores(y_test, preds, 'mean'))
overall_mark

,type,precision,recall,f_score,roc auc
0,mean,0.641577,0.730612,0.683206,0.949837


__Задание 1.__ Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [29]:
def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

In [30]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
X = pd.merge(user_embeddings, target, 'left')
X.head(1)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.368476,0.0,0.0,0.0,0.0,0.012605,0.112228,0.044379,0.0,...,0.0,0.0,0.0,0.0,0.0,0.134971,0.0,0.0,0.036871,0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [32]:
%%time
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

Wall time: 82.6 ms


LogisticRegression()

In [33]:
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.08605873, 0.04408281, 0.45582441, 0.41419276, 0.09909754,
       0.05016984, 0.11930435, 0.11042165, 0.36541606, 0.05924199])

In [34]:
overall_mark = overall_mark.append(get_scores(y_test, preds, 'median'))
overall_mark

,type,precision,recall,f_score,roc auc
0,mean,0.641577,0.730612,0.683206,0.949837
0,median,0.743902,0.746939,0.745418,0.966042


__Задание 2.__ Повторить, но используя уже не медиану, а max

In [35]:
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [36]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
X = pd.merge(user_embeddings, target, 'left')
X.head(1)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.581042,0.0,0.110094,0.152473,0.106308,0.143115,0.28042,0.14494,0.161569,...,0.0,0.244939,0.125997,0.108587,0.0,0.42238,0.135843,0.0,0.05351,0


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [38]:
%%time
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

Wall time: 104 ms


LogisticRegression()

In [39]:
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([5.06838902e-02, 2.43961162e-04, 7.65943845e-01, 5.98251314e-02,
       3.09812946e-02, 8.48638165e-02, 7.20277358e-02, 1.29211778e-01,
       6.21663123e-03, 5.29162721e-01])

In [40]:
overall_mark = overall_mark.append(get_scores(y_test, preds, 'max'))

__Задание 3.__ Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max по метрикам roc_auc, precision, recall, f_score. 

In [41]:
overall_mark

,type,precision,recall,f_score,roc auc
0,mean,0.641577,0.730612,0.683206,0.949837
0,median,0.743902,0.746939,0.745418,0.966042
0,max,0.738776,0.738776,0.738776,0.956653


__Задание 4.__ Сделать выводы.

Лучший результат дает медиана, что неудивительно, так как это самый надежный вариант для анализа данных. Медиана менее подвержена шумам и выбросам. Roc_auc всегда дает очень высокую оценку, что на мой взгляд не очень хорошо. Удивительно, что max показывает себя лучше, чем mean. Я такого не ожидал. В целом все три метода справились хорошо, но лучше всего использовать медиану. (Или tf_idf, но я его не сделал, времени не осталось уже)